# Computational Theory
**Ethan Conneely (G00393941)**

Imports used for countdown

`random` was used for generating random games of countdown for the solver to solve.

`itertools` was used for getting the permutations for all the possible combinations of operations on the numbers.

In [1]:
import random
import itertools

# Game Rules

Here I have defined the rules for a valid game of countdown it can be used to generate a random game to test the solver on it.

The rules for the game are as follows:

- The player may choose the number of large numbers (25,50,75,100) to be used in the game 0-4.
    - The game then picel the number of large number from that list there will not be duplicates
- The remaining number are picked from a selection of 20 number from 1-10 each number is double up in the selection



In [2]:
def generate_game(bigNumbers) -> tuple[list[int], int]:
    if bigNumbers > 4 or bigNumbers < 0:
        raise ValueError("Invalid bigNumbers")

    largeNumbers = random.sample([25, 50, 75, 100], bigNumbers)
    smallNumbers = random.sample(list(range(1, 11)) + list(range(1, 11)), 6 - bigNumbers)

    numbers = largeNumbers + smallNumbers

    return (numbers, random.randint(101, 999))

In [3]:
operations = {
    "+": lambda x, y: x + y,
    "-": lambda x, y: x - y,
    "*": lambda x, y: x * y,
    "/": lambda x, y: x // y,  # integer division
}

In [4]:
def recusive_solve_countdown(numbers, target, calculations=[]):
    solutions = []

    # iterate over all pair of number pairs
    for pair in itertools.combinations(numbers, 2):
        # Remove the pair from the list of numbers and make copy
        computedNumbers = numbers.copy()
        computedNumbers.remove(pair[0])
        computedNumbers.remove(pair[1])

        for op, calculate in operations.items():
            a = max(pair)
            b = min(pair)

            if (a == 0 or b == 0) or (a % b != 0) and op == "/":
                continue  # skip division by zero and none whole number division

            global computation_count
            computation_count+=1

            val = calculate(a, b)

            calcs = [*calculations, (a, op, b, val)]

            if val == target:
                solutions.append(calcs)

            sols = recusive_solve_countdown([val, *computedNumbers], target, calcs)
            if len(sols) != 0:
                solutions = [*sols, *solutions]

    return solutions

In [5]:
def math_expression(expr, expresions):
    left = expresions.get(expr[0])
    if left is None:
        left = expr[0]
    else:
        del expresions[expr[0]]
        left = math_expression(left, expresions)

    right = expresions.get(expr[2])
    if right is None:
        right = expr[2]
    else:
        del expresions[expr[2]]
        right = math_expression(right, expresions)

    return f"({left} {expr[1]} {right})"

In [6]:
def get_expression_lookup(solution):
    expressions = {}
    for expr in solution:
        expressions[expr[3]] = (expr[0], expr[1], expr[2])
    return expressions

In [7]:
unique_expressions = []

def print_solutions(solutions):
    unique_expressions.clear()
    for solution in solutions:
        expressions = get_expression_lookup(solution)
        expr_paren = math_expression(solution[-1], expressions)

        # remove expressions that result in the same expression
        if expr_paren in unique_expressions:
            continue

        unique_expressions.append(expr_paren)

        for expr in solution:
            print(f"{expr[0]} {expr[1]} {expr[2]} = {expr[3]}")

        print(expr_paren)
        print()

    print("Unique expressions: ", len(unique_expressions))
    print("Found expressions: ", len(solutions))

In [8]:
computation_count=0

## Single Solution

In [10]:
game = ([25, 50, 5, 9, 3, 10], 899)
computation_count=0
solutions = recusive_solve_countdown(game[0], game[1])
print_solutions(solutions)
print(f"Computations: {computation_count:,}")

50 * 9 = 450
450 - 3 = 447
447 * 10 = 4470
4470 + 25 = 4495
4495 / 5 = 899
(((((50 * 9) - 3) * 10) + 25) / 5)

Unique expressions:  1
Found expressions:  1
Computations: 1,418,717


## Fixed Game

In [9]:
game = ([75, 50, 2, 3, 7], 103)
computation_count=0
solutions = recusive_solve_countdown(game[0], game[1])
print_solutions(solutions)
print(f"Computations: {computation_count:,}")

50 * 7 = 350
75 - 3 = 72
350 / 2 = 175
175 - 72 = 103
(((50 * 7) / 2) - (75 - 3))

50 * 7 = 350
350 / 2 = 175
175 + 3 = 178
178 - 75 = 103
((((50 * 7) / 2) + 3) - 75)

50 * 7 = 350
350 / 2 = 175
175 - 75 = 100
100 + 3 = 103
((((50 * 7) / 2) - 75) + 3)

50 - 3 = 47
75 * 2 = 150
150 - 47 = 103
((75 * 2) - (50 - 3))

50 / 2 = 25
75 - 3 = 72
25 * 7 = 175
175 - 72 = 103
(((50 / 2) * 7) - (75 - 3))

50 / 2 = 25
75 + 3 = 78
78 + 25 = 103
((75 + 3) + (50 / 2))

50 / 2 = 25
25 * 7 = 175
175 + 3 = 178
178 - 75 = 103
((((50 / 2) * 7) + 3) - 75)

50 / 2 = 25
25 * 7 = 175
175 - 75 = 100
100 + 3 = 103
((((50 / 2) * 7) - 75) + 3)

50 / 2 = 25
25 + 3 = 28
75 + 28 = 103
(75 + ((50 / 2) + 3))

50 / 2 = 25
75 + 25 = 100
100 + 3 = 103
((75 + (50 / 2)) + 3)

50 * 2 = 100
75 * 7 = 525
100 + 3 = 103
((50 * 2) + 3)

75 * 2 = 150
150 + 3 = 153
153 - 50 = 103
(((75 * 2) + 3) - 50)

75 * 2 = 150
150 - 50 = 100
100 + 3 = 103
(((75 * 2) - 50) + 3)

Unique expressions:  13
Found expressions:  25
Computations: 26,17

## Random Game

In [22]:
game = generate_game(3)

print("numbers", game[0])
print("target", game[1])
print()

computation_count=0
solutions = recusive_solve_countdown(game[0], game[1])
print_solutions(solutions)
print(f"Computations: {computation_count:,}")

numbers [75, 100, 50, 7, 3, 5]
target 621

7 - 3 = 4
100 * 5 = 500
75 + 50 = 125
500 - 4 = 496
496 + 125 = 621
(((100 * 5) - (7 - 3)) + (75 + 50))

7 - 3 = 4
100 * 5 = 500
75 + 50 = 125
125 - 4 = 121
500 + 121 = 621
((100 * 5) + ((75 + 50) - (7 - 3)))

7 - 3 = 4
100 * 5 = 500
75 + 50 = 125
500 + 125 = 625
625 - 4 = 621
(((100 * 5) + (75 + 50)) - (7 - 3))

7 - 3 = 4
100 * 5 = 500
50 - 4 = 46
500 + 75 = 575
575 + 46 = 621
(((100 * 5) + 75) + (50 - (7 - 3)))

7 - 3 = 4
100 * 5 = 500
50 - 4 = 46
75 + 46 = 121
500 + 121 = 621
((100 * 5) + (75 + (50 - (7 - 3))))

7 - 3 = 4
100 * 5 = 500
50 - 4 = 46
500 + 46 = 546
546 + 75 = 621
(((100 * 5) + (50 - (7 - 3))) + 75)

7 - 3 = 4
100 * 5 = 500
75 - 4 = 71
500 + 50 = 550
550 + 71 = 621
(((100 * 5) + 50) + (75 - (7 - 3)))

7 - 3 = 4
100 * 5 = 500
75 - 4 = 71
71 + 50 = 121
500 + 121 = 621
((100 * 5) + ((75 - (7 - 3)) + 50))

7 - 3 = 4
100 * 5 = 500
75 - 4 = 71
500 + 71 = 571
571 + 50 = 621
(((100 * 5) + (75 - (7 - 3))) + 50)

7 - 3 = 4
100 * 5 = 500
